<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Activity" data-toc-modified-id="Activity-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Activity</a></span></li><li><span><a href="#Libraries" data-toc-modified-id="Libraries-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Setting-up-the-data" data-toc-modified-id="Setting-up-the-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Setting up the data</a></span></li><li><span><a href="#Modeling-changes-to-a-tissue-during-infection" data-toc-modified-id="Modeling-changes-to-a-tissue-during-infection-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Modeling changes to a tissue during infection</a></span></li><li><span><a href="#See-the-change" data-toc-modified-id="See-the-change-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>See the change</a></span></li><li><span><a href="#How-immune-cells-affect-infection" data-toc-modified-id="How-immune-cells-affect-infection-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>How immune cells affect infection</a></span></li><li><span><a href="#Over-time" data-toc-modified-id="Over-time-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Over time</a></span></li><li><span><a href="#Plotting" data-toc-modified-id="Plotting-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Plotting</a></span></li></ul></div>

# Activity

    
    Imagine a tissue with n x n cells with different states:
    
    H: Healthy
    VA: Viral assembly
    VR: Viral release
    D: Dead
    
    INFECT_PROB = probability of infection (i.e. the likelihood of a cell becoming infected)
    
    

# Libraries

In [39]:
import matplotlib.pyplot
import pandas as pd
import numpy as np
import random
import plotly as py
import plotly.graph_objs as go

# Setting up the data

In [88]:
INFECT_PROB = 0.5 

n = 30 # size of tissue
current = pd.DataFrame("H", index = range(n), columns = range(n)) # write empty data table

max_starting_infected = 5 # choose maximum number of starting infected cells
n_starting_infected = random.randint(0, max_starting_infected) # choose random number of starting infected cells

points = set()
while len(points) < n_starting_infected:
    points.add((random.randint(0, n-1), random.randint(0, n-1))) # choose (x,y) where to place the infected cells in tissue

print("# starting infected cells: {}".format(n_starting_infected))

points

# starting infected cells: 2


{(15, 22), (18, 7)}

In [89]:
# set these points to 1 for "infected"
for point in points:
    current.iloc[point[0], point[1]] = "VR"

# Modeling changes to a tissue during infection

    Next stages for:
    
        Healthy cell: 
            If adjacent to an infectious cell, give a random probability between 0 and 1.
            If probability <= INFECT_PROB, healthy cell is infected and becomes H -> VA
            
        Viral assembly:
            Release of viruses (i.e. VA -> VR)
            
        Viral release:
            Dies (i.e. VR -> D)
            
        Dead cell:
            No change
            
            

In [90]:
def one_time_step(current, infect_prob):
    
    updated = current.copy()
    for row in range(n):
        for col in range(n):

            # make changes to healthy cell
            if current.iloc[row, col] == "H":
                if row != n-1:
                    if current.iloc[row+1,col] == "VR":
                        if random.random() < infect_prob:
                            updated.iloc[row,col] = "VA"

                if col != n-1:       
                    if current.iloc[row,col+1] == "VR":
                        if random.random() < infect_prob:
                            updated.iloc[row,col] = "VA"

                if row != 0:
                    if current.iloc[row-1,col] == "VR":
                        if random.random() < infect_prob:
                            updated.iloc[row,col] = "VA"

                if col != 0:
                    if current.iloc[row,col-1] == "VR":
                        if random.random() < infect_prob:
                            updated.iloc[row,col] = "VA"

            # make changes to newly infected cell
            if current.iloc[row, col] == "VA":
                updated.iloc[row, col] = "VR"
            
            # make changes to infectious cell 
            if current.iloc[row, col] == "VR":
                updated.iloc[row, col] = "D"
    
            # make changes to dead cell
            if current.iloc[row, col] == "D":
                updated.iloc[row, col] = "D" 
                
    return(updated)
            
def t_time_step(t, current, infect_prob):
    
    out = []
    out.append(current.values)
    
    updated = current.copy()
    for step in range(t):
        updated = one_time_step(updated, infect_prob)
        out.append(updated.values)
        
    return(out)
        
def heatmap():
    pass

# See the change

In [91]:
n_time_steps = 50
res = t_time_step(n_time_steps, current, infect_prob = INFECT_PROB) 

In [92]:
heatmaps=res

In [93]:
numerify = {'H': 0, 'VA': 0.25, 'VR': 0.5, 'D': 0.75}

In [94]:
for t in range(len(heatmaps)):
    for i in range(heatmaps[t].shape[0]):
        for j in range(heatmaps[t].shape[1]):
            tmp = numerify[heatmaps[t][i][j]]
            heatmaps[t][i][j] = tmp

In [103]:
legend_colors = [
    [0, 'blue'],
    [0.25, 'blue'],
    [0.25, 'red'],
    [0.5, 'red'],
    [0.5, 'green'],
    [0.75, 'green'],
    [0.75, 'yellow'],
    [1.0, 'yellow']
]

numbers = [0, 0.25, 0.5, 0.75]
names = ['Healthy', 'Viral Assembly', 'Viral Release', 'Dead']

In [104]:
fig = go.Figure(data=go.Heatmap(z=heatmaps[0], colorscale=legend_colors, colorbar=dict(tickvals=numbers, ticktext=names, thickness=25)),
               frames=[go.Frame(data=go.Heatmap(z=heatmaps[i])) for i in range(len(res))])
fig.update_layout(
    updatemenus=[
        dict(type="buttons", visible=True,
        buttons=[dict(label="Play", method="animate", args=[None])]
            )])
fig.show()

# How immune cells affect infection

Add immune system idea 
- First stage infected cells are killed before infecting others with some probability
- Autophagy turns 1s and 2s into 3s
- Add an immune cell state that can potentially move (randomly)
- Have infectious process separate from immune cells

Probability associated with number of starting infected cells

healthy, infectious, newly infected

In [167]:
n = 10
n_starting_infected = random.randint(0, 5)
n_starting_immune = random.randint(0, 10)
current = pd.DataFrame('H', index = range(n), columns = range(n))

In [168]:
# Add infected cells
infected_points = set()
while len(infected_points) < n_starting_infected:
    infected_points.add((random.randint(0, n-1), random.randint(0, n-1)))
for point in infected_points:
    current.iloc[point[0], point[1]] = 'VR'

In [169]:
# Add immune cells
immune_points = set()
while len(immune_points) < n_starting_immune:
    immune_points.add((random.randint(0, n-1), random.randint(0, n-1)))
for point in immune_points:
    current.iloc[point[0], point[1]] = 'I'

In [170]:
INFECT_PROB = 0.8

def one_time_step(current, infect_prob):
    
    updated = current.copy()
    for row in range(n):
        for col in range(n):

            if current.iloc[row, col] == 'H':
                if row != n-1:
                    if current.iloc[row+1,col] == 'VR':
                        if random.random() < infect_prob:
                            updated.iloc[row,col] = 'VA'

                if col != n-1:       
                    if current.iloc[row,col+1] == 'VR':
                        if random.random() < infect_prob:
                            updated.iloc[row,col] = 'VA'

                if row != 0:
                    if current.iloc[row-1,col] == 'VR':
                        if random.random() < infect_prob:
                            updated.iloc[row,col] = 'VA'

                if col != 0:
                    if current.iloc[row,col-1] == 'VR':
                        if random.random() < infect_prob:
                            updated.iloc[row,col] = 'VA'

            if current.iloc[row, col] == 'VA':
                updated.iloc[row, col] = 'VR'

            if current.iloc[row, col] == 'VR':
                #probability
                updated.iloc[row, col] = 'D'

            if current.iloc[row, col] == 'D':
                updated.iloc[row, col] = 'D' 
                
    return(updated)  

In [171]:
def immune_time_step(current):
    updated = current.copy()
    for row in range(n):
        for col in range(n):
            if current.iloc[row, col] == 'I':
                if row != n-1:
                    if current.iloc[row+1,col] in ['VA','VR']:
                        updated.iloc[row+1,col] = 'D'

                if col != n-1:       
                    if current.iloc[row,col+1]in ['VA','VR']:
                        updated.iloc[row,col+1] = 'D'

                if row != 0:
                    if current.iloc[row-1,col] in ['VA','VR']:
                        updated.iloc[row-1,col] = 'D'

                if col != 0:
                    if current.iloc[row,col-1] in ['VA','VR']:
                        updated.iloc[row,col-1] = 'D'
    return(updated)

In [172]:
def t_time_step(t, current, infect_prob):
    
    out = []
    
    out.append(current.values)
    
    updated = current.copy()
    for step in range(t):
        updated = immune_time_step(updated)
        updated = one_time_step(updated, infect_prob)
        out.append(updated.values)
        
    return(out)
        
def heatmap():
    pass
        
# if there is a any cell that is infectious, then there is some probability that it becomes infeccted (50%)

In [173]:
(current == 'VR').sum().sum()

4

# Over time

In [174]:
n_time_steps = 50
res = t_time_step(n_time_steps, current, infect_prob = INFECT_PROB) 

In [175]:
heatmaps=res

In [176]:
numerify = {'H': 0, 'VA': 0.2, 'VR': 0.4, 'D': 0.6, 'I': 0.8}

In [177]:
for t in range(len(heatmaps)):
    for i in range(heatmaps[t].shape[0]):
        for j in range(heatmaps[t].shape[1]):
            tmp = numerify[heatmaps[t][i][j]]
            heatmaps[t][i][j] = tmp

In [178]:
legend_colors = [
    [0, 'blue'],
    [0.2, 'blue'],
    [0.2, 'red'],
    [0.4, 'red'],
    [0.4, 'green'],
    [0.6, 'green'],
    [0.6, 'yellow'],
    [0.8, 'yellow'],
    [0.8, 'purple'],
    [1.0, 'purple']
]

numbers = [0, 0.2, 0.4, 0.6, 0.8]
names = ['Healthy', 'Viral Assembly', 'Viral Release', 'Dead', 'Immune']

In [179]:
fig = go.Figure(data=go.Heatmap(z=heatmaps[0], colorscale=legend_colors, colorbar=dict(tickvals=numbers, ticktext=names, thickness=25)),
               frames=[go.Frame(data=go.Heatmap(z=heatmaps[i])) for i in range(len(res))])
fig.update_layout(
    updatemenus=[
        dict(type="buttons", visible=True,
        buttons=[dict(label="Play", method="animate", args=[None])]
            )])
fig.show()

In [20]:
res = t_time_step(100, current, infect_prob = INFECT_PROB)

In [21]:
heatmaps=res

In [22]:
fig = go.Figure(data=go.Heatmap(z=heatmaps[0]),
               frames=[go.Frame(data=go.Heatmap(z=heatmaps[i])) for i in range(len(res))])
fig.update_layout(
    updatemenus=[
        dict(type="buttons", visible=True,
        buttons=[dict(label="Play", method="animate", args=[None])]
            )])
fig.show()

# Plotting